In [1]:
import pandas as pd
import json
import music21
from __future__ import print_function

import pretty_midi
import pypianoroll

import numpy as np
import pickle
import pandas as pd
import scipy.stats
from collections import Counter
from functools import reduce
import os
from copy import deepcopy
import sys
import argparse
import logging
import coloredlogs
import logging
import warnings
import multiprocessing as mp

from track_separate import predict

logger = logging.getLogger(__name__)
warnings.filterwarnings("ignore")

In [4]:
running_dir = './midi-miner'
melody_model = pickle.load(open(running_dir + '/melody_model','rb'))
bass_model = pickle.load(open(running_dir+ '/bass_model','rb'))
chord_model = pickle.load(open(running_dir+ '/chord_model','rb'))
drum_model = pickle.load(open(running_dir + '/drum_model', 'rb'))
logger.handlers = []

console = logging.StreamHandler()
console.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s : %(levelname)s : %(message)s',
                              datefmt='%Y-%m-%d %H:%M:%S')
console.setFormatter(formatter)
logger.addHandler(console)

coloredlogs.install(level='INFO', logger=logger, isatty=True)

In [30]:
for folder in range(0, 10):
    folder = str(folder)
    output_folder = f'../data/output/{folder}'

    output_json_name = os.path.join(output_folder, "files_result.json")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder, exist_ok=True)

    logfile = output_folder + '/track_separate.log'
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO,
                    datefmt='%Y-%m-%d %H:%M:%S', filename=logfile)
    
    def filter_by_melody(file):
        input_folder = '.'
        file_name = f'../data/lmd_full/{folder}/{file}'

        if len(file_name) > 0:
            all_names = [file_name]
            input_folder = os.path.dirname(file_name)
        else:
            all_names = walk(input_folder)

        all_file_prog = predict(all_names, input_folder, output_folder, 'melody',
                                melody_model, bass_model, chord_model, drum_model)

        with open(os.path.join(output_folder,'program_result.json'), 'w') as fp:
            json.dump(all_file_prog, fp, ensure_ascii=False)
    
    p = mp.Pool(12)
    result = p.map(func=filter_by_melody, iterable=os.listdir(f'../data/lmd_full/{folder}'))
    p.close()
    p.join()

In [31]:
for folder in ['b', 'c', 'd', 'e', 'f']:
    output_folder = f'../data/output/{folder}'

    output_json_name = os.path.join(output_folder, "files_result.json")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder, exist_ok=True)

    logfile = output_folder + '/track_separate.log'
    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO,
                    datefmt='%Y-%m-%d %H:%M:%S', filename=logfile)
    
    def filter_by_melody(file):
        input_folder = '.'
        file_name = f'../data/lmd_full/{folder}/{file}'

        if len(file_name) > 0:
            all_names = [file_name]
            input_folder = os.path.dirname(file_name)
        else:
            all_names = walk(input_folder)

        all_file_prog = predict(all_names, input_folder, output_folder, 'melody',
                                melody_model, bass_model, chord_model, drum_model)

        with open(os.path.join(output_folder,'program_result.json'), 'w') as fp:
            json.dump(all_file_prog, fp, ensure_ascii=False)
    
    p = mp.Pool(12)
    result = p.map(func=filter_by_melody, iterable=os.listdir(f'../data/lmd_full/{folder}'))
    p.close()
    p.join()